# BE instationnaire - 0 - Préliminaires

Ce notebook montre la mise en place d'un calcul avec l'utilisation d'un certain nombre de librairies `python` et `ISAE/DAEP`. Dans les notebooks suivants, un certain nombre d'actions seront encapsulées dans un fichier à importer

## mise en forme

Les commandes suivantes gèrent la mise en forme du notebook et paramètrent la sorties des figures et vidéos.

In [ ]:
from IPython.core.display import HTML ; HTML(open("./custom2.css", "r").read()) # notebook style

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi']  = 80
plt.rcParams['savefig.dpi'] = 100
#plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above, uses JavaScript
plt.rcParams["animation.html"] = "html5" # for matplotlib 2.0 and below, converts to x264 using ffmpeg video codec

## Librairie de calcul `pyfvm`

En plus du standard de calcul `numpy`, on importe ici les différents sous-modules de la librairie `pyfvm`:
* `mesh`:  objets maillage
* `field`: objets champs résultat et son maillage, permet la visualisation rapide
* `xnum`: choix des méthodes numériques pour la discrétisation spatiale
* `integration`: choix des méthodes d'intégration temporelle
* `modelphy.euler` :  modèle physique des équations d'Euler 1D
* `modeldisc`: définition du second membre discret à partir du maillage, du modèle physique, des conditions aux limites et des méthodes numériques

In [ ]:
# librairies Python
import time
import numpy as np 
# librairies ISAE/DAEP
import pyfvm.mesh           as msh
import pyfvm.field          as sol
import pyfvm.xnum           as numx
import pyfvm.integration    as numt
import pyfvm.modelphy.euler as euler
import pyfvm.modeldisc      as modeldisc
#
nx       = 200
meshsim  = msh.unimesh(ncell=nx,  length=10.)
model = euler.model()
bcsym = { 'type': 'sym'}
bcper = { 'type': 'per'}

rhs = modeldisc.fvm(model, meshsim, numx.muscl(numx.vanalbada), bcL=bcsym, bcR=bcsym)
solver = numt.rk3ssp(meshsim, rhs)

## 1. Simulation du cas de calcul

Les commandes suivantes gèrent la mise en forme du notebook et paramètrent la sorties des figures et vidéos.

### 1.1 définition de la condition initiale

On travaille avec des variables normalisées.

<p class="question">
1. Calculer la vitesse du son de l'écoulement moyen pour cet écoulement normalisé.
</p>
<p class="answer">    réponse ! </p>

<p class="question">
2. Quel temps de simulation est nécessaire pour la propagation d'une onde acoustique sur la longueur du tube $10m$
</p>
<p class="answer">    réponse ! </p>

### 1.2 exécution de la simulation

Les commandes suivantes gèrent la mise en forme du notebook et paramètrent la sorties des figures et vidéos.

In [ ]:
# computation
#
nsol    = 120
endtime = 12.
cfl     = .5

start = time.clock()
fsol = solver.solve(finit, cfl, np.linspace(0., endtime, nsol+1))
cputime = time.clock()-start

print "cpu time computation (",solver.nit,"it) :",cputime,"s"
print "  %.2f µs/cell/it"%(cputime*1.e6/solver.nit/meshsim.ncell)

## Exploitation des résultats

Les commandes suivantes gèrent la mise en forme du notebook et paramètrent la sorties des figures et vidéos.

In [ ]:
# Figure / Plot of final results
#
varname='velocity'
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(12,4))
ax1.set_ylabel(varname) ; ax1.set_ylim(-vavg-vmag, vavg+vmag)
ax1.grid(linestyle='--', color='0.5')
line1, = fsol[-1].plot(varname, 'k-', axes=ax1)
ax2.set_ylabel('t') ; ax2.set_xlim(0., 10.)
#ax2.grid(linestyle='--', color='0.5')
ttime = [ fsol[i].time for i in range(nsol+1) ]
xx, xt = np.meshgrid(xc, ttime)
solgrid = [ fsol[i].phydata(varname) for i in range(nsol+1) ]
#error = [ np.sqrt(np.sum((fsol[i].phydata(varname)-mach_th)**2)/nx)/mach_th for i in range(nsol+1) ]
#print error
flood  = ax2.contour(xx, xt, solgrid, np.linspace(-vavg-vmag, vavg+vmag, 50))
line2, = ax2.plot([0., 10.], [ttime[-1], ttime[-1]], 'k--')


In [ ]:
import matplotlib.animation as anim
#
def animate(k):
    #i = min(k, nsol)
    fsol[k].set_plotdata(line1, 'mach')
    line2.set_data([0., 10.], [ttime[k], ttime[k]])
    return line1, line2

ani = anim.FuncAnimation(fig=fig, func=animate, frames=range(nsol+1), interval=100, blit=True)
ani
#from IPython.display import HTML
#HTML(ani.to_html5_video()) # if no rcparams